In [ ]:
import pandas as pd

In [ ]:
import seaborn as sns

In [ ]:
path = "/tmp/results2.csv"
path = "results-ntasks-laptop.csv"
path = "results-periods-laptop.csv"

In [ ]:
df = pd.read_csv(path, sep=";")

In [ ]:
df

In [ ]:
df.groupby(["nbt", "use_case"])["U"].count()

In [ ]:
df["duration_s"] = df["duration_ns"] / 1e9

In [ ]:
x = "U"
x = "nbt"
x = "max_period"

In [ ]:
g = sns.scatterplot(data=df, x=x, y="duration_s", hue="use_case")

In [ ]:
g = sns.lineplot(data=df, x=x, y="duration_s", hue="use_case")
# g.set(yscale='log')

In [ ]:
g = sns.lineplot(data=df, x=x, y="visited_count", hue="use_case")
g.set(yscale='log')